In this notebook we will explore how to 'broadcast' controllers on robots in our simulator.

### V-REP setup

By using the "respondable" mask for entities in V-REP we can let objects pass through each other, as documented [here.](http://www.coppeliarobotics.com/helpFiles/en/shapeDynamicsProperties.htm)

In this setup we have 5 robots stacked on top of each other, and we run different controllers on each robot to efficiently search some sort of controller space.

To run mulitple controllers simultaneously, we use the python `multiprocess` module. Each process requires communication to V-REP, so we need to open more remoteAPI socket servers. In this scenario we opened ports 19990 - 19996, in `V-REP/remoteApiConnections.txt`:
```
//central control
portIndex1_port 		= 19990
portIndex1_debug 		= false
portIndex1_syncSimTrigger 	= true

//rob0
portIndex2_port 		= 19991
portIndex2_debug 		= false
portIndex2_syncSimTrigger 	= true

//rob1
portIndex3_port 		= 19992
portIndex3_debug 		= false
portIndex3_syncSimTrigger 	= true

portIndex4_port 		= 19993
portIndex4_debug 		= false
portIndex4_syncSimTrigger 	= true

portIndex5_port 		= 19994
portIndex5_debug 		= false
portIndex5_syncSimTrigger 	= true

portIndex6_port 		= 19995
portIndex6_debug 		= false
portIndex6_syncSimTrigger 	= true

portIndex7_port 		= 19996
portIndex7_debug 		= false
portIndex7_syncSimTrigger 	= true
```

### Controller Setup

First we setup the machinery to run a controller.

In [1]:
# Cross-notebook include shim
with open("nbinclude.ipynb") as nbinclude_f: # don't rename nbinclude_f
    import IPython.nbformat.current
    get_ipython().run_cell(IPython.nbformat.current.read(nbinclude_f, 'json').worksheets[0].cells[0].input)
nbinclude('Narwhal')

/usr/local/lib/python2.7/dist-packages/IPython/nbformat/current.py:19: UserWarning: IPython.nbformat.current is deprecated.

- use IPython.nbformat for read/write/validate public API
- use IPython.nbformat.vX directly to composing notebooks of a particular version

  """)


Each robot is connected on its own remote API socket.

However we need 1 socket to do the high level orchestration of the simulation: checking that the sensors have started reading, and actually automatically starting and stopping the simulation.

In [2]:
cid = vrep.simxStart('127.0.0.1',19990,True,True,5000,5)

In this demonstration we are only using the left and right motor throttles as parameters.

In [5]:
lefts = [10, 10, 10, 10, 10]
rights = [10, 14, 16, 20, 22]

These other parameters are used to setup the simulations and record data from the run.

In [101]:
ports = [19991, 19992, 19993, 19994, 19995]
nids = [0,1,2,3,4]
narwhals = [Narwhal(cid, n) for n in nids]
dicts = [{} for n in nids]

`do_driveby` is the control loop. It runs the motors at the throttles `left` and `right` while visual servoing on the robot perimeter collumns.

In [102]:
def do_driveby(cid, nid, left, right, ret_dict):
    if cid == -1:
        return
    narwhal = Narwhal(cid, nid)
    traj_data = np.ndarray(shape=(5000, 5))
    images = np.ndarray(shape=(5000, 128*3))
    
    img = np.array(narwhal.get_linescan())
    center = 128.*3./2.
    
    narwhal.drive(left, right)
    t0 = time.time()
    t = time.time() - t0
    for i in xrange(traj_data.shape[0]):

        t = time.time() - t0

        # process image 
        img = np.array(narwhal.get_linescan())
        com = scipy.ndimage.measurements.center_of_mass(img)[0]
        if math.isnan(com):
            com = center

        # act
        error = center - com
        vel = .1 * error
        narwhal.set_neck_vel(vel)

        #record state
        x,y,z = narwhal._get_position('zumo')
        angle = narwhal.get_neck_pos()
        traj_data[i] = t, x, y, angle, vel
        if img.shape[0] == images.shape[1]:
            images[i] = img

        time.sleep(.001)
    ret_dict['traj']=traj_data
    ret_dict['images']=images
    ret_dict['cid']=cid
    ret_dict['nid']=nid
    ret_dict['left']=left
    ret_dict['right']=right

### Broadcasting Controllers Serially

The following code block serially runs each controller serially.

In [111]:
# Stop all robots.
vrep.simxStopSimulation(cid, vrep.simx_opmode_oneshot_wait)
for n in narwhals:
    n.drive(0,0)

# Start simulations
vrep.simxStartSimulation(cid, vrep.simx_opmode_oneshot_wait)
for i in xrange(len(nids)):
    
    # run experiment
    do_driveby(cid, nids[i], lefts[i], rights[i], dicts[i])
    
    # stop
    time.sleep(.1)
    narwhals[i].drive(0,0)
    narwhals[i].set_neck_vel(0)

vrep.simxStopSimulation(cid, vrep.simx_opmode_oneshot_wait)

0

##### Saving data to disk

When we're done we can save all the data to disk for offline inspection.

In [104]:
import pickle
f = open('driveby_broadcast_serial.pkl', 'w')
pickle.dump(dicts, f)
f.close()

In [105]:
!du -h driveby_broadcast_serial.pkl

297M	driveby_broadcast_serial.pkl


In [108]:
!gzip driveby_broadcast_serial.pkl

gzip: driveby_broadcast_serial.pkl: No such file or directory


In [107]:
!du -h driveby_broadcast_serial.pkl.gz

768K	driveby_broadcast_serial.pkl.gz


### Broadcasting Controllers in Parallel

Now, let's try to run all controllers simultaneously.

In [10]:
from multiprocessing import Process, Manager, Event, Lock

In [90]:
manager = Manager()
pardicts = [manager.dict() for i in xrange(len(nids))]
start_event = Event()

In [91]:
""" We need to wrap the `do_driveby` because our simulations now
run on separate processes. """
def do_driveby_multiproc(port, nid, left, right, ret_dict, start_event):
    cid = vrep.simxStart('127.0.0.1',port,True,True,5000,5)
    if cid == -1:
        return
    start_event.wait()
    do_driveby(cid, nid, left, right, ret_dict)
    vrep.simxFinish(cid)

In [92]:
processes = []
for i in xrange(len(nids)):
    processes.append(Process(target=do_driveby_multiproc, args=(
                                                    ports[i],
                                                    nids[i],
                                                    lefts[i],
                                                    rights[i],
                                                    pardicts[i],
                                                    start_event)))

In [93]:
vrep.simxStartSimulation(cid, vrep.simx_opmode_oneshot_wait)
for process in processes:
    process.start()
start_event.set()
for process in processes:
    process.join()
vrep.simxStopSimulation(cid, vrep.simx_opmode_oneshot_wait)

/usr/lib/python2.7/dist-packages/scipy/ndimage/measurements.py:1271: RuntimeWarning: invalid value encountered in double_scalars
  for dir in range(input.ndim)]
/usr/lib/python2.7/dist-packages/scipy/ndimage/measurements.py:1271: RuntimeWarning: invalid value encountered in double_scalars
  for dir in range(input.ndim)]
/usr/lib/python2.7/dist-packages/scipy/ndimage/measurements.py:1271: RuntimeWarning: invalid value encountered in double_scalars
  for dir in range(input.ndim)]
/usr/lib/python2.7/dist-packages/scipy/ndimage/measurements.py:1271: RuntimeWarning: invalid value encountered in double_scalars
  for dir in range(input.ndim)]
/usr/lib/python2.7/dist-packages/scipy/ndimage/measurements.py:1271: RuntimeWarning: invalid value encountered in double_scalars
  for dir in range(input.ndim)]


0

##### Saving data to disk

In [ ]:
dictscopied = []
for d in pardicts:
    dictscopied.append(d.copy())

In [95]:
import pickle
f = open('driveby_broadcast_parallel.pkl', 'w')
pickle.dump(dictscopied, f)
f.close()

In [96]:
!du -h driveby_broadcast_parallel.pkl

297M	driveby_broadcast_parallel.pkl


In [109]:
!gzip driveby_broadcast_parallel.pkl

In [110]:
!du -h driveby_broadcast_parallel.pkl.gz

772K	driveby_broadcast_parallel.pkl.gz
